In [1]:
%pip install torch torchvision

Note: you may need to restart the kernel to use updated packages.


In [2]:
%conda install pytorch torchvision -c pytorch

Channels:
 - pytorch
 - defaults
Platform: osx-arm64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [12]:
%pip install opencv-python

  Using cached opencv_python-4.11.0.86-cp37-abi3-macosx_13_0_arm64.whl.metadata (20 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-macosx_13_0_arm64.whl (37.3 MB)
Note: you may need to restart the kernel to use updated packages.


In [28]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from PIL import Image

# Paths to dataset
dataset_path = "/Users/user/Documents/I5/AI/Final_AI_Project/split_dataset"
train_path = f"{dataset_path}/train"
val_path = f"{dataset_path}/val"
test_path = f"{dataset_path}/test"

# Data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}

# Load datasets
train_dataset = datasets.ImageFolder(train_path, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(val_path, transform=data_transforms['val'])
test_dataset = datasets.ImageFolder(test_path, transform=data_transforms['test'])

print(f"Classes: {train_dataset.classes}")
print(f"Training samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}, Test samples: {len(test_dataset)}")

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

# Load pre-trained ResNet model
pretrained_model = models.resnet50(pretrained=True)
num_classes = len(train_dataset.classes)
pretrained_model.fc = nn.Linear(pretrained_model.fc.in_features, num_classes)

# Freeze all layers except the final fully connected
for param in pretrained_model.parameters():
    param.requires_grad = False
for param in pretrained_model.fc.parameters():
    param.requires_grad = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pretrained_model = pretrained_model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(pretrained_model.fc.parameters(), lr=0.001)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    pretrained_model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = pretrained_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum().item()
        total += labels.size(0)

        if i % 10 == 0:
            print(f"Batch {i}/{len(train_loader)}, Loss: {loss.item():.4f}")

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}, Accuracy: {correct / total * 100:.2f}%")

torch.save(pretrained_model.state_dict(), "resnet50_vehicle_classifier.pth")
print("Model saved.")

# Prediction
def predict_image(model, image_path, class_names):
    model.eval()
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = outputs.max(1)
        return class_names[predicted.item()]

test_image_path = "/Users/user/Documents/I5/AI/Final_AI_Project/sample_image.jpg"
if os.path.exists(test_image_path):
    predicted_class = predict_image(pretrained_model, test_image_path, train_dataset.classes)
    print(f"Predicted class: {predicted_class}")
else:
    print(f"Test image path does not exist: {test_image_path}")


Classes: ['airplane', 'bicycles', 'cars', 'motorbikes', 'ships']
Training samples: 8908, Validation samples: 1908, Test samples: 1911
Batch 0/279, Loss: 1.5686
Batch 10/279, Loss: 0.8651
Batch 20/279, Loss: 0.4970
Batch 30/279, Loss: 0.3024
Batch 40/279, Loss: 0.3242
Batch 50/279, Loss: 0.2031
Batch 60/279, Loss: 0.3176
Batch 70/279, Loss: 0.0878
Batch 80/279, Loss: 0.2787
Batch 90/279, Loss: 0.1989
Batch 100/279, Loss: 0.2269
Batch 110/279, Loss: 0.3177
Batch 120/279, Loss: 0.1463
Batch 130/279, Loss: 0.0935
Batch 140/279, Loss: 0.0978
Batch 150/279, Loss: 0.2177
Batch 160/279, Loss: 0.0571
Batch 170/279, Loss: 0.0622
Batch 180/279, Loss: 0.1350
Batch 190/279, Loss: 0.0860
Batch 200/279, Loss: 0.2817
Batch 210/279, Loss: 0.1546
Batch 220/279, Loss: 0.3634
Batch 230/279, Loss: 0.1808
Batch 240/279, Loss: 0.0953
Batch 250/279, Loss: 0.2917
Batch 260/279, Loss: 0.1382
Batch 270/279, Loss: 0.1400
Epoch 1/10, Loss: 0.2481, Accuracy: 93.20%
Batch 0/279, Loss: 0.1319
Batch 10/279, Loss: 0.12

In [8]:
from torchvision import models, datasets
import torch.nn as nn
import torch

# Reload class names dynamically
train_dataset_path = "/Users/user/Documents/I5/AI/Final_AI_Project/split_dataset/train"
train_dataset = datasets.ImageFolder(train_dataset_path)
class_names = train_dataset.classes
num_classes = len(class_names)

# Reload the model
pretrained_model = models.resnet50(pretrained=False)  # Set pretrained to False
pretrained_model.fc = nn.Linear(pretrained_model.fc.in_features, num_classes)

# Load trained weights
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pretrained_model.load_state_dict(torch.load("resnet50_vehicle_classifier.pth", map_location=device))
pretrained_model = pretrained_model.to(device)

print("Model loaded successfully.")


Model loaded successfully.


/var/folders/rh/pb3vmf1d43d_z4xt5jr0bmwr0000gn/T/ipykernel_46444/1703366878.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_model.load_state_dict(torch.load(

In [41]:
accuracy = correct / total * 100
print(f"Accuracy: {accuracy:.2f}%")

Accuracy: 96.65%


In [ ]:
import matplotlib.pyplot as plt

# Assuming you have stored the loss and accuracy values during training
epochs = list(range(1, num_epochs + 1))
train_losses = []  # Fill this with the training loss values
train_accuracies = []  # Fill this with the training accuracy values

# Plotting the loss
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss over Epochs')
plt.legend()

# Plotting the accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracies, label='Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training Accuracy over Epochs')
plt.legend()

plt.show()